# Barrier method

<i>minimize</i> $f(x)=-\sum_{i=1}^{n}log(x_i)$ <br>
<i>subject to</i> $Ax=b$ <br>
<i>additional constraints:</i> $x\geq{0}$ <br>

Введем логарифмический барьер: <br>
$x\geq{0} ⇒ f_i(x)=1-x_i\leq{0}$ <br>
$\phi(x)=-\frac{1}{t}\sum_{i=1}^{m}log(-f_i(x))=-\frac{1}{t}\sum_{i=1}^{m}log(x_i-1)$

Новая задача:
<i>minimize</i> $f_b(x)=-\sum_{i=1}^{n}log(x_i)-\frac{1}{t}\sum_{i=1}^{m}log(x_i-1)$ <br>
<i>subject to</i> $Ax=b$ <br>

$f(x)$ и $\phi(x)$ являются выпуклыми как суммы выпуклых функций, аналогично $f_b(x) = f(x)+\phi(x)$ является выпуклой при фиксированном $t>0$. Ограничение $Ax=b$ задаёт аффинное множество, которое всегда является выпуклым. Таким образом, задача является **выпуклой**.